In [10]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten,Conv2D,Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.utils import to_categorical

In [56]:
df = pd.read_csv("C:/Users/Harvey/Desktop/Yelp_data_set/restuarant_review_V4.csv")

In [57]:
df.head()

,Unnamed: 0,sentiment,stars,Processed_Reviews
0,0,1,5,go lunch steak sandwich delicious caesar salad...
1,1,1,4,ill first admit excite go la tavolta food snob...
2,2,0,1,place go hill clearly cut back staff food qua...
3,3,1,4,like walk back time every saturday morning sis...
4,4,0,1,walk around 4 friday afternoon sit table bar w...


In [58]:
df.drop("Unnamed: 0",axis = 1,inplace = True)
df.drop("sentiment",axis = 1, inplace = True)

In [59]:
df.head()

,stars,Processed_Reviews
0,5,go lunch steak sandwich delicious caesar salad...
1,4,ill first admit excite go la tavolta food snob...
2,1,place go hill clearly cut back staff food qua...
3,4,like walk back time every saturday morning sis...
4,1,walk around 4 friday afternoon sit table bar w...


In [101]:
train = df.sample(frac = 0.8,random_state = 200)
test = df.drop(train.index)

In [102]:
train['Processed_Reviews'][1]

'ill first admit excite go la tavolta food snob group friends suggest go dinner look online menu nothing special seem overprice  im also big order pasta go alas outnumber thank goodness order sea bass special die cook perfectly season perfectly perfect portion say enough good things dish server ask seem proud dish say  doesnt chef incredible job   hubby get crab tortellini also love hear mmmm good around table waiter super nice even give us free desserts last people restaurant service slow place pack jug wine large group good conversation didnt seem bother anyone   order calamari fry zucchini appetizers leave mussels   sea bass special, highly recommend it. chicken parm crab tortellini also good big. chicken romano bite bland. house salads teeny.   make reservation still expect wait food. go large group people plan loud. go date unless fight feel like hear anything say.  ask sit side room available.'

In [103]:
train.loc[train['stars'] == 4,'stars'] = 3
train.loc[train['stars'] == 5,'stars'] = 4
train.loc[:,'stars'] -= 1
print(np.unique(train['stars']))

[0 1 2 3]


In [104]:
train['Processed_Reviews'][1]

'ill first admit excite go la tavolta food snob group friends suggest go dinner look online menu nothing special seem overprice  im also big order pasta go alas outnumber thank goodness order sea bass special die cook perfectly season perfectly perfect portion say enough good things dish server ask seem proud dish say  doesnt chef incredible job   hubby get crab tortellini also love hear mmmm good around table waiter super nice even give us free desserts last people restaurant service slow place pack jug wine large group good conversation didnt seem bother anyone   order calamari fry zucchini appetizers leave mussels   sea bass special, highly recommend it. chicken parm crab tortellini also good big. chicken romano bite bland. house salads teeny.   make reservation still expect wait food. go large group people plan loud. go date unless fight feel like hear anything say.  ask sit side room available.'

In [105]:
import numpy as np
test = np.array([1,2,3,4,5])
print(test-1)
print(test.shape)
print(np.arange(test.shape[0]))
test = test -1
print(to_categorical(test,num_classes = 5))

[0 1 2 3 4]
(5,)
[0 1 2 3 4]
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


In [106]:
train['Processed_Reviews'][1]

'ill first admit excite go la tavolta food snob group friends suggest go dinner look online menu nothing special seem overprice  im also big order pasta go alas outnumber thank goodness order sea bass special die cook perfectly season perfectly perfect portion say enough good things dish server ask seem proud dish say  doesnt chef incredible job   hubby get crab tortellini also love hear mmmm good around table waiter super nice even give us free desserts last people restaurant service slow place pack jug wine large group good conversation didnt seem bother anyone   order calamari fry zucchini appetizers leave mussels   sea bass special, highly recommend it. chicken parm crab tortellini also good big. chicken romano bite bland. house salads teeny.   make reservation still expect wait food. go large group people plan loud. go date unless fight feel like hear anything say.  ask sit side room available.'

In [107]:
max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train['Processed_Reviews'])
list_tokenized_train = tokenizer.texts_to_sequences(train['Processed_Reviews'])

maxlen = 130
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = to_categorical(train['stars'])


In [109]:
print(y[:5,])
print(train['stars'][:5])

[[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
152464    3
176685    2
272019    3
66031     2
448757    3
Name: stars, dtype: int64


In [111]:
embed_size = 128
model = Sequential()
model.add(Embedding(max_features, embed_size,input_length = maxlen))
model.add(Conv1D(filters = 50, kernel_size = 2, padding='same',activation = "relu"))
model.add(GlobalMaxPool1D())
#model.add(Flatten())
model.add(Dense(4, activation="softmax"))
model.summary()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 130, 128)          768000    
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 130, 50)           12850     
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 50)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 204       
Total params: 781,054
Trainable params: 781,054
Non-trainable params: 0
_________________________________________________________________


In [112]:
batch_size = 100
epochs = 3
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 328264 samples, validate on 82066 samples
Epoch 1/3
328264/328264 [==============================] - 159s 485us/step - loss: 0.6804 - acc: 0.6917 - val_loss: 0.6263 - val_acc: 0.7147
Epoch 2/3
328264/328264 [==============================] - 159s 484us/step - loss: 0.5885 - acc: 0.7346 - val_loss: 0.6230 - val_acc: 0.7162
Epoch 3/3
328264/328264 [==============================] - 160s 486us/step - loss: 0.5486 - acc: 0.7550 - val_loss: 0.6317 - val_acc: 0.7122


In [ ]:
##################################################
# Base line CNN using pretrained word vectors 
